<a href="https://colab.research.google.com/github/thaisstein/Simulation-Project/blob/main/SimulationProjectThais_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing and importing libraries

In [1]:
!pip install simpy

In [3]:
import sys
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import math
import pandas as pd
import simpy
import queue

## Creating packet as a tuple with its creation time (t), its sequence number(seqno), its packet size (pktSize)
and the identifier of the source (ident)

In [4]:
class packet(object):
  def __init__(self, t, ident, pktSize):
    global seqno # global counter that increments when a packet is created
    seqno += 1
    self.t = t
    self.ident = ident
    self.pktSize = pktSize
    self.seqno = seqno
    packet_arrival_times.append(t)
    print(f"Packet arrived at time {t}, Sequence Number: {seqno}, Source ID: {ident}")


## Defining the queue

In [12]:
class queueClass(object): # buffer + server
  def __init__(self, env, queueCapa, serviceRate):
    self.env = env
    self.inService = 0
    self.buffer = queue.Queue(maxsize = queueCapa)
    self.queueCapacity = queueCapa
    self.serviceRate = serviceRate
    self.lastChange = 0
    self.response_times = {1: [], 2: [], 3: []}  # Average response times for each source

  def service(self, source):
    p = self.buffer.get() # extract packages from buffer
    print(f"Service started for packet {p.seqno} from Source {source} at time {self.env.now}")
    self.inService = 1 # service in process
    service_start_times.append(self.env.now) # time the service i started
    service_time = 1/self.serviceRate # random service time from an exponencial distribution
    # of a mean 1.0, used to model time between events in a Poisson process, service time for the packet
    yield self.env.timeout(service_time)
    response_time = self.env.now - p.t
    self.response_times[source].append(response_time)
    del p

    print(f"Service completed at time {self.env.now}")
    if self.queueLength > 0: # still packets in the queue - recursion
      self.env.process(self.service())
    else:
      self.inService = 0

  def calculate_average_response_time(self):
    for source, response_times in self.response_times.items():
        if len(response_times) > 0:
            average_response_time = sum(response_times) / len(response_times)
            print(f"Average Response Time for Source {source}: {average_response_time} seconds")

  def reception(self, source, pkt):
    self.buffer.put(pkt)
    if self.inService == 0:
      print(f"Packet added to the queue at time {self.env.now}, Sequence Number: {pkt.seqno}")
      self.env.process(self.service(pkt.ident))


## Defining Data Traffic as a Poisson source

In [6]:
class dataTraffic(object):
  def __init__(self, env, rate, q, ident, pktSize):
    self.env = env
    self.rate = rate
    self.q = q
    self.ident = ident
    self.pktSize = pktSize

    self.nbEmissions= 0
    self.queueLosses = 0
    self.action = env.process(self.run())

  def run(self):
    while True:
      poisson_arrival_time = np.random.exponential(1.0/self.rate) # Poisson
      yield self.env.timeout(poisson_arrival_time)

      packet_lengths = [50, 500, 1500]  # in bytes
      packet_probs = [0.4, 0.3, 0.3]
      pktSize = np.random.choice(packet_lengths, p=packet_probs)
      p = packet(self.env.now, self.ident, self.pktSize)

      self.q.reception(self, p)
      self.nbEmissions += 1


## Defining Voice Traffic as a constant source

In [7]:
class voiceTraffic(object):
  def __init__(self, env, rate, q, ident, pktSize):
    self.env = env
    self.rate = rate
    self.q = q
    self.ident = ident
    self.pktSize = pktSize

    self.nbEmissions= 0
    self.queueLosses = 0
    self.cpterPrintLR = 0

    self.action = env.process(self.run())

  def run(self):
    while True:
      constant_arrival_time = 1 / (self.rate / (self.pktSize * 8)) # Constant
      yield self.env.timeout(constant_arrival_time)
      p = packet(self.env.now, self.ident, self.pktSize)
      self.q.reception(self, p)
      self.nbEmissions += 1


## Defining Video Traffic as highly bursty ON/OFF source exponencially distributed.

In [8]:
class videoTraffic(object):
  def __init__(self, env, rate, q, ident, pktSize):
    self.env = env
    self.rate = rate
    self.q = q
    self.pktSize = pktSize
    self.ident = ident

    self.nbEmissions= 0
    self.queueLosses = 0
    self.cpterPrintLR = 0
    self.transmission_time = 30e6  # Adjust the peak rate as needed

    self.ton =  1e-3  # average duration of "ON" periods
    self.toff = np.random.exponential(0.2)  # average duration of "OFF" periods
    self.action = env.process(self.run())
    self.burstiness = 0

  def run(self):
    while True:
      yield self.env.timeout(self.ton)
      yield self.env.timeout(transmission_time)
      p = packet(self.env.now,t,self.ident, self.pktSize)
      self.q.reception(self, p)
      self.nbEmissions += 1
      seqno += 1

      yield self.env.timeout(self.toff)
      self.burstiness = ((on_duration + off_duration) / on_duration)
      self.peak_rate = burstiness * rate
      # Calculate the time needed to transmit the packet at the peak rate
      response_time = (self.pktSize * 8 / self.peak_rate) # bytes to bits

      print("Burstiness is", burstiness)


## Running

In [13]:
simulationDuration = 10
seqno = 0
np.random.seed(10)
packet_arrival_times = []
service_start_times = []
queueCapa = float('inf') # infinite
serviceRate = 100e6 # 100 Mbps

env = simpy.Environment()
q = queueClass(env,queueCapa,serviceRate)
s1 = dataTraffic(env,30e6,q,1,1) #pkt size changes
s2 = voiceTraffic(env,20e6,q,100,2)
s3 = videoTraffic(env,30e6,q,1000,3)

env.run(until=simulationDuration)

queue.calculate_average_response_time()


# Plot service start times, packet arrival times, and loss rate on the same graph
#plt.plot(packet_arrival_times, color='blue', alpha=0.7, label='Packet Arrival')
#plt.plot(service_start_times,color='red', alpha=0.7, label='Service Start')
#plt.axvline(loss_rate, color='green', linestyle='dashed', linewidth=2, label=f'Loss Rate: {loss_rate:.2f}')

#plt.xlabel('Time')
#plt.ylabel('Frequency')
#plt.title('Packet Arrival, Service Start')
#plt.grid(True)
#plt.legend()
#plt.show()

Packet arrived at time 6.990099034630943e-10, Sequence Number: 1, Source ID: 1
Packet added to the queue at time 6.990099034630943e-10, Sequence Number: 1
Service started for packet 1 from Source 1 at time 6.990099034630943e-10


AttributeError: ignored



For each average response time, that is related to a b value, you have to calculate the epsilont and then the epsilonT